In [2]:
import tkinter as tk
from tkinter import filedialog

import scipy.io
import os
import numpy as np

import gc

from funcs import vhdr_to_mne, bipolar_montage, align, prepare_data, concatenation,load_sorciere_regressors, model_definition, trf_spyeeg

## paths

In [3]:
# first 'select a folder direction 'patient espace''
# then, select the sorciere file in the NAS (sorcière = file.vhdr)
# then, select the pianos files in the NAS (pianos = [file.vhdr, file.vhdr..]
# Fonction pour sélectionner le chemin vers le dossier "sorciere"
def select_sorciere_path():
    global sorciere_path
    sorciere_path = filedialog.askopenfilenames()
    if sorciere_path:
        print("Fichiers sélectionnés:")
        for file in sorciere_path:
            print(file)

#  Fonction pour sélectionner le chemin vers le dossier "piano"
def select_piano_path():
    global piano_path
    piano_path = filedialog.askopenfilenames()
    if piano_path:
        print("Fichiers sélectionnés:")
        for file in piano_path:
            print(file)


# Fonction pour sélectionner le chemin vers le dossier de sortie
def select_out_path():
    global out_path
    out_path = filedialog.askdirectory()
    if out_path:
        print("Chemin vers le dossier de sortie sélectionné:", out_path)

# Création de la fenêtre principale
root = tk.Tk()
root.title("Ouvrir")

# Déclaration des variables pour stocker les chemins sélectionnés
sorciere_path = ""
piano_path = ""
out_path = ""

# Boutons pour sélectionner les chemins vers les dossiers
sorciere_button = tk.Button(root, text="Select Sorcière *.vhdr (on NAS)", command=select_sorciere_path)
sorciere_button.pack(pady=5)

piano_button = tk.Button(root, text="Select Pianos *.vhdr (on NAS)", command=select_piano_path)
piano_button.pack(pady=5)

out_button = tk.Button(root, text="Select patient workspace (locally)", command=select_out_path)
out_button.pack(pady=5)

root.mainloop()

# Vous pouvez maintenant utiliser les variables sorciere_path, piano_path et out_path dans votre application.


Fichiers sélectionnés:
//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Sorcière/250403B-C_0013.vhdr
Fichiers sélectionnés:
//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano/250403B-C_0004.vhdr
//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano/250403B-C_0006.vhdr
//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano/250403B-C_0008.vhdr
//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano/250403B-C_0010.vhdr
Chemin vers le dossier de sortie sélectionné: C:/Users/nadege/Desktop/camille/seeg/agnes/Nic_Ele


## Import .vhdr to mne

In [4]:
sorciere_mne = vhdr_to_mne(sorciere_path)
pianos_mne = vhdr_to_mne(piano_path)

//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Sorcière/250403B-C_0013.vhdr
250403B-C_0013.vhdr
Extracting parameters from //139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Sorcière\250403B-C_0013.vhdr...


Setting channel info structure...
Reading 0 ... 294911  =      0.000 ...   575.998 secs...
//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano/250403B-C_0004.vhdr
250403B-C_0004.vhdr
Extracting parameters from //139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano\250403B-C_0004.vhdr...
Setting channel info structure...
Reading 0 ... 51199  =      0.000 ...    99.998 secs...


c:\Users\nadege\Desktop\camille\seeg\agnes\funcs.py:35: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_fif.append(mne.io.read_raw_brainvision(os.path.join(path, file), preload=True))


//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano/250403B-C_0006.vhdr
250403B-C_0006.vhdr
Extracting parameters from //139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano\250403B-C_0006.vhdr...
Setting channel info structure...
Reading 0 ... 65535  =      0.000 ...   127.998 secs...


c:\Users\nadege\Desktop\camille\seeg\agnes\funcs.py:35: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_fif.append(mne.io.read_raw_brainvision(os.path.join(path, file), preload=True))


//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano/250403B-C_0008.vhdr
250403B-C_0008.vhdr
Extracting parameters from //139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano\250403B-C_0008.vhdr...
Setting channel info structure...
Reading 0 ... 93183  =      0.000 ...   181.998 secs...


c:\Users\nadege\Desktop\camille\seeg\agnes\funcs.py:35: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_fif.append(mne.io.read_raw_brainvision(os.path.join(path, file), preload=True))


//139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano/250403B-C_0010.vhdr
250403B-C_0010.vhdr
Extracting parameters from //139.124.148.123/seeg/Seeg_1998-2025/Seeg2025/sEEG_11_Nic_Ele/03.04/Piano\250403B-C_0010.vhdr...
Setting channel info structure...
Reading 0 ... 29695  =      0.000 ...    57.998 secs...


c:\Users\nadege\Desktop\camille\seeg\agnes\funcs.py:35: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_fif.append(mne.io.read_raw_brainvision(os.path.join(path, file), preload=True))
c:\Users\nadege\Desktop\camille\seeg\agnes\funcs.py:35: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_fif.append(mne.io.read_raw_brainvision(os.path.join(path, file), preload=True))


In [5]:
piano_path, file = os.path.split(piano_path[0])
sorciere_path, file = os.path.split(sorciere_path[0])
del file

In [6]:
stim_piano = [file for file in os.listdir(piano_path+'/stim') if os.path.isfile(os.path.join(piano_path+'/stim', file)) and file.endswith('.mp3')]
stim_sorcière = [file for file in os.listdir(sorciere_path+'/stim') if os.path.isfile(os.path.join(sorciere_path+'/stim', file)) and file.endswith('.mp3')]

In [7]:
stim_sorcière

['sorciere_2ms_trig.mp3']

## Montage bipolaire

Do a bipolar montage en soustrayant l'activité de deux shaft consécutives /!\ de le même electrode

In [8]:
sorciere_bipol, channels_bipol = bipolar_montage(sorciere_mne[0], chans=True)
sorciere_bipol = [sorciere_bipol]

pianos_bipol = []
for piano in pianos_mne :
    pianos_bipol.append(bipolar_montage(piano, chans=False))

In [9]:
#Trier les channels dans l'ordre alphabétique et numérique
def custom_sort(value):
    # Séparer la partie alphabétique et numérique
    alpha_part = ''.join(filter(lambda x: x.isalpha() or x == "'", value))
    num_part = int(''.join(filter(str.isdigit, value)))
    return alpha_part, num_part

sorted_channels_bipol = sorted(channels_bipol, key=custom_sort)

# Sorciere
sorciere_bipol_sorted = [np.empty_like(sorciere_bipol[0])]
for i, channel_name in enumerate(sorted_channels_bipol):
    channel_index = channels_bipol.index(channel_name)
    sorciere_bipol_sorted[0][:, i] = sorciere_bipol[0][:, channel_index]

# Pianos
pianos_bipol_sorted = []
for piano in pianos_bipol :
    sorted_piano_bipol = np.empty_like(piano)
    for i, channel_name in enumerate(sorted_channels_bipol):
        channel_index = channels_bipol.index(channel_name)
        sorted_piano_bipol[:, i] = piano[:, channel_index]
    pianos_bipol_sorted.append(sorted_piano_bipol)

del sorted_piano_bipol

In [10]:
scipy.io.savemat(out_path+'/channels.mat', {'ch':np.array(sorted_channels_bipol)})

In [11]:
import mne
from pydub import AudioSegment
from collections import Counter
import matplotlib.pyplot as plt
from scipy.signal import correlate

def align_origin(raw_fif, raw_array, stim_dir) :
  stim_name = os.path.basename(stim_dir)

  #load .mp3
  aud = AudioSegment.from_file(stim_dir, format="mp3")
  fs_stim = aud.frame_rate

  # stereo format : audio/trigger
  chans = aud.split_to_mono()
  trigg = chans[1].get_array_of_samples()  # triggers

  # binarized triggers
  thrsh_exceed_idx = []
  seuil = 3000
  for idx, valeur in enumerate(trigg):
    if valeur > seuil and trigg[idx - 1] < seuil and trigg[idx - 2] < seuil:
      thrsh_exceed_idx.append(idx)

  trigg_stim = np.array(thrsh_exceed_idx)
  trigg_stim = trigg_stim / fs_stim

  # load data
  fs_data = raw_fif.info['sfreq']

  # find events
  orig_events, events_dict = mne.events_from_annotations(raw_fif)
  #take only trigg of interest
  counter = Counter(orig_events[:, 2])
  t_int = counter.most_common(1)[0][0]
  trigg_data_samples = np.squeeze(orig_events[np.where(orig_events[:, 2] == t_int), 0])
  #trigg_data_samples = trigg_data_samples[5:]
  trigg_data_secondes = trigg_data_samples / fs_data

  plt.figure()
  plt.vlines(trigg_data_secondes, 0,1, label = "trigg_data")
  plt.vlines(trigg_stim, 1,2, color='orange', label = "trigg_stim")
  plt.xlim(0,10)
  plt.legend()
  plt.title("before alignement")

  print('data :', len(trigg_data_secondes))
  print('stim :', len(trigg_stim))

  i=0
  max_iteration = 150
  while True:
    if len(trigg_data_secondes) == len(trigg_stim):
      print('same numbre of trigger')
      trigg_stim_zero = trigg_stim - trigg_stim[0]
      trigg_data_zero = trigg_data_secondes - trigg_data_secondes[0]

    elif len(trigg_data_secondes) < len(trigg_stim):
      print('< number of trigger')
      if i >= len(trigg_stim):  # Si i dépasse la taille de trigg_stim
          print(f"Stopping: i ({i}) exceeded trigg_stim size ({len(trigg_stim)})")
          break
      trigg_stim_zero = trigg_stim - trigg_stim[i]
      trigg_data_zero = trigg_data_secondes - trigg_data_secondes[0]

    elif len(trigg_data_secondes) > len(trigg_stim):
      print('> number of trigger')
      if i >= len(trigg_data_secondes):  # Si i dépasse la taille de trigg_data_secondes
          print(f"Stopping: i ({i}) exceeded trigg_data_secondes size ({len(trigg_data_secondes)})")
          break
      trigg_stim_zero = trigg_stim - trigg_stim[0]
      trigg_data_zero = trigg_data_secondes - trigg_data_secondes[i]

    # calcul delay between all triggs
    matrice_differences = np.zeros((len(trigg_data_zero), len(trigg_stim_zero)))
    for j in range(len(trigg_data_zero)):
      for k in range(len(trigg_stim_zero)):
        matrice_differences[j, k] = trigg_data_zero[j] - trigg_stim_zero[k]

    # take only closest pairs of trigger
    if len(trigg_data_zero) <= len(trigg_stim_zero):
      trigg_stim_zero_syncr = []
      for j in range(len(trigg_data_zero)):
        idx_syncr = np.argmin(np.abs(matrice_differences[j, :]))
        trigg_stim_zero_syncr.append(trigg_stim_zero[idx_syncr])
      # calcul delay between closest pairs
      matrice_differences_ = np.zeros((len(trigg_data_zero), len(trigg_stim_zero_syncr)))
      for j in range(len(trigg_data_zero)):
        for k in range(len(trigg_stim_zero_syncr)):
          matrice_differences_[j, k] = trigg_data_zero[j] - trigg_stim_zero_syncr[k]
      # calcul delay between closest pairs
      matrice_differences_ = np.zeros((len(trigg_data_zero), len(trigg_stim_zero_syncr)))
      for j in range(len(trigg_data_zero)):
        for k in range(len(trigg_stim_zero_syncr)):
          matrice_differences_[j, k] = trigg_data_zero[j] - trigg_stim_zero_syncr[k]

    elif len(trigg_data_zero) > len(trigg_stim_zero):
      trigg_data_zero_syncr = []
      for j in range(len(trigg_stim_zero)):
        idx_syncr = np.argmin(np.abs(matrice_differences[:, j]))
        trigg_data_zero_syncr.append(trigg_data_zero[idx_syncr])
      # calcul delay between closest pairs
      matrice_differences_ = np.zeros((len(trigg_data_zero_syncr), len(trigg_stim_zero)))
      for j in range(len(trigg_data_zero_syncr)):
        for k in range(len(trigg_stim_zero)):
          matrice_differences_[j, k] = trigg_data_zero_syncr[j] - trigg_stim_zero[k]
      # calcul delay between closest pairs
      matrice_differences_ = np.zeros((len(trigg_data_zero_syncr), len(trigg_stim_zero)))
      for j in range(len(trigg_data_zero_syncr)):
        for k in range(len(trigg_stim_zero)):
          matrice_differences_[j, k] = trigg_data_zero_syncr[j] - trigg_stim_zero[k]

    # mean delay
    tmp = np.mean(np.diagonal(matrice_differences_))
    print(np.abs(round(tmp, 4)))
    if np.abs(round(tmp, 4)) > 0.005 :
      i+=1
      continue

    else :
      if i == 0 :
        raw_array_ = raw_array[trigg_data_samples[0]:trigg_data_samples[-1],:]
        plt.figure()
        plt.vlines(trigg_data_zero, 0,1, label = "trigg_data")
        plt.vlines(trigg_stim_zero, 1,2, color='orange', label = "trigg_stim")
        plt.xlim(-1,10)
        plt.legend()
        plt.title("after alignement")
      else :
        if len(trigg_data_secondes) < len(trigg_stim):
          raw_array_ = raw_array[trigg_data_samples[0]:trigg_data_samples[-1],:]
          zrs = int((trigg_stim[i]-trigg_stim[0])*fs_data)
          add_zeros = np.zeros((zrs, raw_array_.shape[1]))
          raw_array_ = np.concatenate((add_zeros, raw_array_), axis=0)

        elif len(trigg_data_secondes) > len(trigg_stim):
          raw_array_ = raw_array[trigg_data_samples[i]:trigg_data_samples[-1],:]
      break

  return raw_array_

## Align

In [12]:
%matplotlib qt
sorciere_aligned = align(raw_fif=sorciere_mne[0], raw_array=sorciere_bipol_sorted[0], stim_dir=os.path.join(sorciere_path,'stim', stim_sorcière[0]))

pianos_aligned=[]
for i in range(len(pianos_bipol_sorted)):
    pianos_aligned.append(align(raw_fif=pianos_mne[i], raw_array=pianos_bipol_sorted[i], stim_dir=os.path.join(piano_path,'stim', stim_piano[i])))

Used Annotations descriptions: ['Comment/?', 'Comment/D?but', 'Stimulus/S 192', 'Stimulus/S 200']
data : 581
stim : 585
< number of trigger
0.0143
< number of trigger
0.0367
< number of trigger
0.0507
< number of trigger
0.0004
Used Annotations descriptions: ['Comment/?', 'Comment/D?but', 'Stimulus/S 192', 'Stimulus/S 200']
data : 99
stim : 152
< number of trigger
0.0185
< number of trigger
0.001
Used Annotations descriptions: ['Comment/?', 'Comment/D?but', 'Stimulus/S 128', 'Stimulus/S 192', 'Stimulus/S 200']
data : 126
stim : 103
> number of trigger
0.0442
> number of trigger
0.0367
> number of trigger
0.0309
> number of trigger
0.0281
> number of trigger
0.0386
> number of trigger
0.0024
Used Annotations descriptions: ['Comment/?', 'Comment/D?but', 'Stimulus/S 128', 'Stimulus/S 192', 'Stimulus/S 200', 'Stimulus/S 201']
data : 183
stim : 130
> number of trigger
0.0065
> number of trigger
0.0062
> number of trigger
0.0658
> number of trigger
0.012
> number of trigger
0.013
> number of

## Filtering 0.1-8Hz - Z-scoring - Downsampling

In [13]:
high_pass = 0.1
low_pass = 35

fs_data = sorciere_mne[0].info['sfreq']

pianos_downsampled = []

sorciere_ultimate = prepare_data(data= sorciere_aligned, picks=sorted_channels_bipol, high_pass=high_pass, low_pass=low_pass, fs_data=fs_data)
for data in pianos_aligned :
    print(data.shape)
    pianos_downsampled.append(prepare_data(data=data, picks=sorted_channels_bipol, high_pass=high_pass, low_pass=low_pass, fs_data=fs_data))

(50113, 178)
(62530, 178)
(86609, 178)
(29378, 178)


## Concatenate Pianos & Load Pianos regressors

In [14]:
# define and load concatenated data and regressors for piano
pianos_ultimate, pianos_regs_all, pianos_regs_all_labels, fs = concatenation(pianos_downsampled, piano_path, stim_piano)

In [15]:
sorted_channels_bipol

['A1-A2',
 'A2-A3',
 'A3-A4',
 'A4-A5',
 'A5-A6',
 'A6-A7',
 'A7-A8',
 'A8-A9',
 'A9-A10',
 'A10-A11',
 'A11-A12',
 'A12-A13',
 'A13-A14',
 'C1-C2',
 'C2-C3',
 'C3-C4',
 'C4-C5',
 'C5-C6',
 'C6-C7',
 'C7-C8',
 'C8-C9',
 'C9-C10',
 'C10-C11',
 'C11-C12',
 'CC1-CC2',
 'CC2-CC3',
 'CC3-CC4',
 'CC4-CC5',
 'CC5-CC6',
 'CC6-CC7',
 'CC7-CC8',
 'CC8-CC9',
 'CC9-CC10',
 'CC10-CC11',
 'CR1-CR2',
 'CR2-CR3',
 'CR3-CR4',
 'CR4-CR5',
 'CR5-CR6',
 'CR6-CR7',
 'CR7-CR8',
 'CR8-CR9',
 'CR9-CR10',
 'CR10-CR11',
 'FT1-FT2',
 'FT2-FT3',
 'FT3-FT4',
 'FT4-FT5',
 'FT5-FT6',
 'FT6-FT7',
 'FT7-FT8',
 'FT8-FT9',
 'FT9-FT10',
 'FT10-FT11',
 'FT11-FT12',
 'FT12-FT13',
 'FT13-FT14',
 'H1-H2',
 'H2-H3',
 'H3-H4',
 'H4-H5',
 'H5-H6',
 'H6-H7',
 'H7-H8',
 'H8-H9',
 'H9-H10',
 'H10-H11',
 'H11-H12',
 'H12-H13',
 'H13-H14',
 'H14-H15',
 'H15-H16',
 'Ia1-Ia2',
 'Ia2-Ia3',
 'Ia3-Ia4',
 'Ia4-Ia5',
 'Ia5-Ia6',
 'Ia6-Ia7',
 'Ia7-Ia8',
 'Ia8-Ia9',
 'Ia9-Ia10',
 'Ia10-Ia11',
 'Ia11-Ia12',
 'Ia12-Ia13',
 'Ia13-Ia14',
 'Ia14-

## Load Sorcière regressors

In [17]:
import sys
sys.path.append(r"C:\Users\nadege\mambaforge\Lib\site-packages")
import textgrids
print("textgrids importé avec succès.")


textgrids importé avec succès.


In [18]:
sorciere_ultimate, sorciere_regs_all, sorciere_regs_all_label = load_sorciere_regressors(sorciere_path, sorciere_ultimate)

In [19]:
print("Socière regressors labels:")
for idx, label in enumerate(sorciere_regs_all_label):
    print(f"{idx}: {label}")

print("\nPianos regressors labels:")
for idx, label in enumerate(pianos_regs_all_labels):
    print(f"{idx}: {label}")


Socière regressors labels:
0: loudness
1: brightness
2: pitch
3: periodicity
4: roughness
5: spectral_flux
6: der_loudness
7: der_brightness
8: der_pitch
9: der_periodicity
10: der_roughness
11: der_spectral_flux
12: half_rectified_der_loudness
13: half_rectified_der_brightness
14: half_rectified_der_pitch
15: half_rectified_der_periodicity
16: half_rectified_der_roughness
17: half_rectified_der_spectral_flux
18: syllabe_onset
19: onsets
20: entropy
21: surprise

Pianos regressors labels:
0: envelope
1: der_envelope_hr
2: onset
3: surprise
4: surprise_positive


## Define models

In [ ]:
# Sorcière
# Acoustic regressors: loudness, brightness, periodicity + derivatives_hr + onset syllabe
model_acoustic_names = [
    'loudness',
    'brightness',
    'periodicity',
    'half_rectified_der_loudness',
    'half_rectified_der_brightness',
    'half_rectified_der_periodicity',
    'syllabe_onset'
]

# Predictive regressors: acoustic + onsets + entropy
model_predictif_names = [
    'loudness',
    'brightness',
    'periodicity',
    'half_rectified_der_loudness',
    'half_rectified_der_brightness',
    'half_rectified_der_periodicity',
    'syllabe_onset',
    'onsets',
    'entropy'
]

sorciere_regs_acoustic, sorciere_regs_acoustic_label, sorciere_regs_predictive, sorciere_regs_predictive_labels = model_definition(
    sorciere_regs_all,
    sorciere_regs_all_label,
    model_acoustic_names,
    model_predictif_names
)


# Pianos
# Acoustic regressors: envelope, der_envelope_hr, onset
model_acoustic_names = [
    'envelope',
    'der_envelope_hr',
    'onset'
]

# Predictive regressors: acoustic + surprise + surprise_positive
model_predictif_names = [
    'envelope',
    'der_envelope_hr',
    'onset',
    'surprise',
    'surprise_positive'
]

pianos_regs_acoustic, pianos_regs_acoustic_label, pianos_regs_predictive, pianos_regs_predictive_labels = model_definition(
    pianos_regs_all,
    pianos_regs_all_labels,
    model_acoustic_names,
    model_predictif_names
)


## TRF

In [18]:
from datetime import datetime

start_at = datetime.now()

In [19]:
n_folds = 5
tw_trf = [-2, 2]  #in sec

coefs_times = np.linspace(tw_trf[0], tw_trf[1], int(np.diff(tw_trf) * fs + 1))  #get TRF time-axis

alphas = [1e-4, 1e-2, 1e2, 1e4, 1e6, 1e8]

### Sorcière : acoustic TRF

In [20]:
import sys
sys.path.append(r"C:\Users\nadege\mambaforge\Lib\site-packages")
import spyeeg
print("Bibliothèque importée avec succès.")

Bibliothèque importée avec succès.


In [21]:
sorciere_regs_acoustic_scores_cv, sorciere_regs_acoustic_coefs_fit = trf_spyeeg(
                                                    regs = np.squeeze(sorciere_regs_acoustic),
                                                    data = np.squeeze(sorciere_ultimate),
                                                    tmin=tw_trf[0], tmax=tw_trf[1],
                                                    fs=fs,
                                                    alpha=alphas,
                                                    channels=sorted_channels_bipol)


# save r2 in .mat format
scipy.io.savemat(out_path+'/sorciere_acoustic.mat',
                 {'data' : sorciere_ultimate,
                  'channels': sorted_channels_bipol,
                  'r2': sorciere_regs_acoustic_scores_cv,
                  'kernels':sorciere_regs_acoustic_coefs_fit,
                  'fs': fs})

gc.collect()

0

### Sorcière : predictive TRF

In [22]:
sorciere_regs_predictive_scores_cv, sorciere_regs_predictive_coefs_fit = trf_spyeeg(
regs = np.squeeze(sorciere_regs_predictive),
data = np.squeeze(sorciere_ultimate),
tmin=tw_trf[0], tmax=tw_trf[1],
fs=fs,alpha=alphas,channels=sorted_channels_bipol)

# save r2 in .mat format
scipy.io.savemat(out_path+'/sorciere_predictive.mat',
                 {'data' : sorciere_ultimate,
                  'channels': sorted_channels_bipol,
                  'r2': sorciere_regs_predictive_scores_cv,
                  'kernels':sorciere_regs_predictive_coefs_fit,
                  'fs': fs})

gc.collect()

0

### Pianos : acoustic TRF

In [23]:
pianos_regs_acoustic_scores_cv, pianos_regs_acoustic_coefs_fit = trf_spyeeg(
    regs = np.squeeze(pianos_regs_acoustic),
    data = np.squeeze(pianos_ultimate),
    tmin=tw_trf[0], tmax=tw_trf[1],
    fs=fs,
    alpha=alphas,
    channels=sorted_channels_bipol)

# save r2 in .mat format
scipy.io.savemat(out_path+'/pianos_acoustic.mat',
                 {'data' : pianos_ultimate,
                  'channels': sorted_channels_bipol,
                  'r2': pianos_regs_acoustic_scores_cv,
                  'kernels':pianos_regs_acoustic_coefs_fit,
                  'fs': fs})

gc.collect()

0

### Pianos : predictive TRF

In [24]:
pianos_regs_predictive_scores_cv, pianos_regs_predictive_coefs_fit = trf_spyeeg(
    regs = np.squeeze(pianos_regs_predictive),
    data = np.squeeze(pianos_ultimate),
    tmin=tw_trf[0], tmax=tw_trf[1],
    fs=fs,
    alpha=alphas,
    channels=sorted_channels_bipol)


# save r2 in .mat format
scipy.io.savemat(out_path+'/pianos_predictive.mat',
                 {'data' : pianos_ultimate,
                  'channels': sorted_channels_bipol,
                  'r2': pianos_regs_predictive_scores_cv,
                  'kernels':pianos_regs_predictive_coefs_fit,
                  'fs': fs})

gc.collect()

0

### Generate a .txt file with scores 

In [25]:
# Create the full path to the file
filename = os.path.join(out_path, 'scores.txt')

with open(filename, 'w') as file:
    # Write the header with aligned columns
    file.write(f'{"Channel":<10}{"Speech Acoustic (in %)":<25}{"Speech Predictif (in %)":<30}{"Music Acoustic (in %)":<30}{"Music Predictif (in %)":<30}\n')

    # Write data to the file
    for chan, scr1, scr2, scr3, scr4 in zip(sorted_channels_bipol, sorciere_regs_acoustic_scores_cv, sorciere_regs_predictive_scores_cv, pianos_regs_acoustic_scores_cv, pianos_regs_predictive_scores_cv):
        file.write(f'{chan:<15}{round(scr1*100,2):<30}{round(scr2*100,2):<25}{round(scr3*100,2):<30}{round(scr4*100,2):<20}\n')


## Plot results

### R2, PCI & ERP - with matlab

In [26]:
import matlab.engine
from datetime import datetime

print('Start at :', datetime.now())


# Démarrer le moteur MATLAB
eng = matlab.engine.start_matlab()
# run MATLAB script
eng.eval(f"out_path = '{out_path}'; run('C:/Users/nadege/Desktop/camille/seeg/agnes/sEEG_rsqr_pci.m')", nargout=0)
#eng.eval(f"out_path = '{out_path}'; run('//dynaserv/home/trebuchon/Desktop/Python_code/sEEG_rsqr_pci.m')", nargout=0)


print("End at:", datetime.now())

Start at : 2025-04-25 15:43:39.619272
End at: 2025-04-25 15:44:48.550983
